In [13]:
import requests
from bs4 import BeautifulSoup
import json 
import os 
# Base URL of the website
url = 'https://www.lnh.edu.pk/Doctors?spe=837cdba4-bf74-4a77-aa22-f7157c1c2cf9'

# Send a request to the page and get the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links to doctor detail pages
doctor_links = [a.get('href') for a in soup.find_all('a', class_='btn') if a.get('href')]

# List to store the doctor details
doctor_details = []

# Loop through each doctor link
for link in doctor_links:
    doctor_url = 'https://www.lnh.edu.pk' + link
    
    doctor_response = requests.get(doctor_url)
    doctor_soup = BeautifulSoup(doctor_response.content, 'html.parser')
    
    name_tag = doctor_soup.find('h5', class_='name')
    name = name_tag.text.strip() if name_tag else 'N/A'

    qualifications_tag = doctor_soup.find('h5', class_='text-theme-colored')
    qualifications = qualifications_tag.text.strip() if qualifications_tag else 'N/A'

    # Locate the department information after the <br> tag
    department_tag = doctor_soup.find('p', class_='font-14 font-weight-500')
    if department_tag:
        br_tag = department_tag.find('br')
        department = br_tag.next_sibling.strip() if br_tag and br_tag.next_sibling else 'N/A'
    else:
        department = 'N/A'
    
    # Extract clinic details (e.g., location, contact, timings, etc.)
    clinics = []
    
    panels = doctor_soup.find_all('div', class_='panel')
    for panel in panels:
        clinic_name_tag = panel.find('div', class_='panel-title')
        clinic_name = clinic_name_tag.text.strip() if clinic_name_tag else 'N/A'
        
        location_tag = panel.find('span', class_='font-weight-700')
        location = location_tag.find_next_sibling('span').text.strip() if location_tag else 'N/A'

        contact_tag = panel.find('span', text='Contact:')
        contact = contact_tag.find_next_sibling('span').text.strip() if contact_tag else 'N/A'
        
        timings = []
        rows = panel.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            day = cols[0].text.strip() if len(cols) > 0 else 'N/A'
            shift = cols[1].text.strip() if len(cols) > 1 else 'N/A'
            timing = cols[2].text.strip() if len(cols) > 2 else 'N/A'
            timings.append({'day': day, 'shift': shift, 'timing': timing})
        
        clinics.append({
            'clinic_name': clinic_name,
            'location': location,
            'contact': contact,
            'timings': timings
        })
    
    doctor_details.append({
        'name': name,
        'qualifications': qualifications,
        'department': department,
        'clinics': clinics
    })


#save in json 

# with open('doctor_details.json', 'w') as file:
#     json.dump(doctor_details, file, indent=4)

doctor_info = "doctors_info.json"


# save i json format 
if os.path.exists(doctor_info):
    with open(doctor_info, 'r') as json_file:
        existing_data = json.load(json_file)
else:
    existing_data = []

# Append new data to existing data
existing_data.extend(doctor_details)

# Write updated data to the JSON file
with open(doctor_info, 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)



# Print or save the extracted data
for doctor in doctor_details:
    print(json.dumps(doctor , indent=2))



/tmp/ipykernel_10827/1700099393.py:50: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contact_tag = panel.find('span', text='Contact:')


{
  "name": "N/A",
  "qualifications": "FCPS",
  "department": "N/A",
  "clinics": [
    {
      "clinic_name": "Paediatrics Urology And Nephrology Clinic",
      "location": "Liaquat National Hospital",
      "contact": "02134412474, 02134413036",
      "timings": [
        {
          "day": "N/A",
          "shift": "N/A",
          "timing": "N/A"
        },
        {
          "day": "Mon",
          "shift": "Morning",
          "timing": "11:30 am  to  02:00 pm"
        },
        {
          "day": "Thu",
          "shift": "Morning",
          "timing": "11:30 am  to  02:00 pm"
        },
        {
          "day": "Fri",
          "shift": "Afternoon",
          "timing": "02:00 pm  to  04:30 pm"
        },
        {
          "day": "Sat",
          "shift": "Afternoon",
          "timing": "02:00 pm  to  04:29 pm"
        },
        {
          "day": "N/A",
          "shift": "N/A",
          "timing": "N/A"
        },
        {
          "day": "N/A",
          "shift": "

Doctor details saved to doctor_details.json
